In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as ipy
import time
import cProfile


In [2]:
def init_grid() :

    grid = np.zeros((8,8), dtype = np.int)
    for i in range(8) :
        for j in range(8) :
            if (i+j)%2 == 1 :
                if i<3 :
                    grid[i,j] = 1
                if i>4 :
                    grid[i,j] = -1
    return grid

In [3]:
grid = init_grid()


In [4]:
def legal_move(grid,player):

    # All necessary grids
    a_front = np.copy(grid)
    a_front[:-1,:-1] *= grid[1:,1:]
    a_back = np.copy(grid)
    a_back[1:,1:] *= grid[:-1,:-1]
    a_front_2 = np.copy(grid)
    a_front_2[:-2,:-2] *= grid[2:,2:] 
    a_back_2 = np.copy(grid)
    a_back_2[2:,2:] *= grid[:-2,:-2]
    
    b_front = np.copy(grid)
    b_front[:-1,1:] *= grid[1:,:-1]
    b_back = np.copy(grid)
    b_back[1:,:-1] *= grid[:-1,1:]
    b_front_2 = np.copy(grid)
    b_front_2[:-2,2:] *= grid[2:,:-2] 
    b_back_2 = np.copy(grid)
    b_back_2[2:,:-2] *= grid[:-2,2:]
    

    # Capture moves
    # Front = negative and there is a player piece (for player 1) and front 2 = 0
    legal_a_front_2 = (a_front_2 == 0)*(grid*player>abs((player-1)/2))*(a_front < 0)
    legal_b_front_2 = (b_front_2 == 0)*(grid*player>abs((player-1)/2))*(b_front < 0)
    legal_a_back_2 = (a_back_2 == 0)*(grid*player>abs((player+1)/2))*(a_back < 0)
    legal_b_back_2 = (b_back_2 == 0)*(grid*player>abs((player+1)/2))*(b_back < 0)
    
    
    # Pick legal moves and end destination
    
    moves_a_front_2 = (np.argwhere(legal_a_front_2==True))
    moves_a_front_2 = np.hstack((moves_a_front_2,(moves_a_front_2+[2,2])))
    moves_b_front_2 = (np.argwhere(legal_b_front_2==True))
    moves_b_front_2 = np.hstack((moves_b_front_2,(moves_b_front_2+[2,-2])))
    moves_a_back_2 = (np.argwhere(legal_a_back_2==True))
    moves_a_back_2 = np.hstack((moves_a_back_2,(moves_a_back_2+[-2,-2])))
    moves_b_back_2 = (np.argwhere(legal_b_back_2==True))
    moves_b_back_2 = np.hstack((moves_b_back_2,(moves_b_back_2+[-2,2])))
    
    moves = np.vstack((moves_a_front_2, moves_b_front_2, moves_a_back_2, moves_b_back_2))
    move_type = 'cap'
    
    if len(moves)==0:
        # Empty moves
        # Front = 0 and there is a player piece (for player 1)
        #                                              should be >0 for player 1, >1 for player -1
        legal_a_front = (a_front == 0)*(grid*player>abs((player-1)/2)) 
        legal_b_front = (b_front == 0)*(grid*player>abs((player-1)/2)) 
        # Back = 0 and there is a player king piece (for player 1)
        #                                              should be >1 for player 1, >0 for player -1
        legal_a_back = (a_back == 0)*(grid*player>(player+1)/2) 
        legal_b_back = (b_back == 0)*(grid*player>(player+1)/2) 

        # Pick legal moves and end destination
        moves_a_front = (np.argwhere(legal_a_front==True))
        moves_a_front = np.hstack((moves_a_front,(moves_a_front+[1,1])))
        moves_b_front = (np.argwhere(legal_b_front==True))
        moves_b_front = np.hstack((moves_b_front,(moves_b_front+[1,-1])))
        moves_a_back = (np.argwhere(legal_a_back==True))
        moves_a_back = np.hstack((moves_a_back,(moves_a_back+[-1,-1])))
        moves_b_back = (np.argwhere(legal_b_back==True))
        moves_b_back = np.hstack((moves_b_back,(moves_b_back+[-1,1])))
        
        moves = np.vstack((moves_a_front, moves_b_front, moves_a_back, moves_b_back))
        move_type = 'empty'
    

    return moves, move_type
    
    
    


In [5]:
def do_move(grid, move, player):
    piece = grid[move[0],move[1]]
    length = abs(move[0]-move[2])
    grid[move[0],move[1]] = 0
    
    # Make kings
    if move[2]==7 or move[2]==0:
        piece = 2*player
        
    grid[move[2],move[3]] = piece
    if length==2:
        mid = (move[0:2]+move[2:4])//2
        grid[mid[0],mid[1]] = 0
        
def test_move(grid, move, player) :
    length = abs(move[0]-move[2])
    test_grid = np.copy(grid)
    piece = test_grid[move[0],move[1]]
    test_grid[move[0],move[1]] = 0
    # Make kings
    if move[2]==7 or move[2]==0:
        piece = 2*player
        
    test_grid[move[2],move[3]] = piece
    if length==2:
        mid = (move[0:2]+move[2:4])//2
        test_grid[mid[0],mid[1]] = 0
    return np.sum(test_grid)

In [6]:

def check_double_cap(grid, player, depth, moves):
    scores = np.zeros(len(moves))
    move_type = 'cap'
    # If so, check if any of the moves results in a potential double cap
    for i in range(len(moves)):
        temp_grid = np.copy(grid)
        do_move(temp_grid, moves[i], player)

        # Check if double cap is possible
        test_moves, test_move_type = legal_move(temp_grid, player)
        if test_move_type == 'cap':
            # If the check moves are cap type, check if any originate at the end position of previous move
            double_cap_check = [np.all((test_move[0:2])==(moves[i][2:4])) for test_move in test_moves]
            if np.sum(double_cap_check)>0:
                move_type = 'double_cap'

                # If so, do not reduce depth and look for same player
                double_scores  = [pick_move(temp_grid, player, depth)[1] for next_move in moves]
                # Score the moves accordingly
                if player == 1:
                    scores[i] = np.max(double_scores)
                elif player ==-1:
                    scores[i] = np.min(double_scores)
        # If not a double move, just return score 
        else:
            scores[i] = test_move(grid, moves[i], player)
    if move_type == 'double_cap':
        return True, scores
    else:
        return False, scores


def pick_move(grid, player, depth):
    
    # Find all possible legal moves
    moves, move_type = legal_move(grid, player)
    scores = np.zeros(len(moves))
    
    if len(moves) == 0:
        moves = [[0, 0, 0, 0]]
        return(moves, np.ones(len(moves))*np.sum(grid), 'empty', 0)
    
    # If depth == 1, just score all the moves
    if depth == 1:
        
        if move_type == 'cap':
            # Check for double cap
            double_cap = check_double_cap(grid, player, depth, moves)
            if double_cap[0] == True:
                scores = double_cap[1]
                move_type = 'double_cap'
        
        # If not a double cap, just return score
        else:
            for i in range(len(moves)):
                scores[i] = test_move(grid, moves[i], player)

    # If depth != 1, assess one level deeper
    else:
        # Check for a double cap
        if move_type == 'cap':
            double_cap = check_double_cap(grid, player, depth, moves)
            if double_cap[0] == True:
                scores = double_cap[1]
                move_type = 'double_cap'
        else:    
            # If no double cap, calculate at depth -1 for opposing player
            for i in range(len(moves)):
                temp_grid = np.copy(grid)
                do_move(temp_grid, moves[i], player)
                # Calculate opponent best move
                opp_scores = pick_move(temp_grid, -player, depth-1)[1]
                if player == 1:
                    scores[i] = np.min(opp_scores)
                if player == -1:
                    scores[i] = np.max(opp_scores)
    # Determine best move
    if player == 1:
        best_move_arg = np.argmax(scores)
    elif player == -1:
        best_move_arg = np.argmin(scores)
    
    return moves, scores, move_type, best_move_arg
        
    
    
    

In [7]:
# def pick_move(grid, player, depth):
    
#     moves, move_type = legal_move(grid, player)

#     if move_type == 'empty':
#         scores = [assess_move(grid, move, player, depth) for move in moves]
#         if player == 1:
#             arg = np.argmax(scores)
#             score = scores[arg]
#         elif player == -1:
#             arg = np.argmin(scores)
#             score = scores[arg]
#     if move_type == 'cap':
#         scores = [assess_cap(grid, move, player, depth) for move in moves]
#         if player == 1:
#             arg = np.argmax(scores)
#             score = scores[arg]
#         elif player == -1:
#             arg = np.argmin(scores)
#             score = scores[arg]
        
#         # Check if the move results in a double cap
#         temp_grid = np.copy(grid)
#         do_move(temp_grid, moves[arg], player)
#         double_cap_check_moves, double_cap_check_move_type = legal_move(temp_grid, player)
        
        
#         # Check if the next move is a cap type
#         if double_cap_check_move_type == 'cap':
#             # If so, check if it is from the same piece
            
#             double_cap_check = [np.all((new_move[0:2])==(moves[arg][2:4])) for new_move in double_cap_check_moves]
#             if np.sum(double_cap_check)>0:
#                 # If so, register double_cap move type
#                 move_type = 'double_cap'
#     print('check move type:'+move_type)
#     print(np.hstack((moves, np.array(scores).reshape(len(moves),1))))
#     return arg, score, moves[arg], move_type


# def assess_cap(grid, move, player, depth):

#     # Depth 1 doesn't check for double cap for now...
#     if depth == 1:
#         temp_grid = np.copy(grid)
#         do_move(temp_grid, move, player)

#         # Check if double cap is possible
#         moves, move_type = legal_move(temp_grid, player)
#         if move_type == 'cap':
#             double_cap_check = [np.all((next_move[0:2])==(move[2:4])) for next_move in moves]
#             if np.sum(double_cap_check)>0:
#                 move_type = 'double_cap'
#                 # If so, do not reduce depth
#                 scores = [assess_cap(temp_grid, next_move, player, depth) for next_move in moves]
#                 if player == 1:
#                     score = np.max(scores)
#                 elif player ==-1:
#                     score = np.min(scores)
#             else:
#                 score = test_move(grid, move, player)
#         else:
#             score = test_move(grid, move, player)
#         return score
#     else:
    
#         temp_grid = np.copy(grid)
#         do_move(temp_grid, move, player)
#         # Check if double cap is possible
#         moves, move_type = legal_move(temp_grid, player)
#         if move_type == 'cap':
#             double_cap_check = [np.all((next_move[0:2])==(move[2:4])) for next_move in moves]
#             if np.sum(double_cap_check)>0:
#                 move_type = 'double_cap'
#                 # If so, do not reduce depth or change player
#                 scores = [assess_cap(temp_grid, next_move, player, depth) for next_move in moves]
#             else:
#                 # Else, go on to normal assessment
#                 moves, move_type = legal_move(temp_grid, -player)

#         if move_type == 'empty':
#             scores = [assess_move(temp_grid, next_move, -player, depth-1) for next_move in moves]
#         if move_type == 'cap':
#             scores = [assess_cap(temp_grid, next_move, -player, depth-1) for next_move in moves]
            
#         if player == 1:
#             score = np.min(scores)
#         elif player == -1:
#             score = np.max(scores)
#         print('cap')
#         print('player:',player, 'depth:',depth, '\n scores:',scores, 'score',score)
#         return score

# def assess_move(grid, move, player, depth):
#     if depth == 1:
#         score = test_move(grid, move, player)
#         return score
        
#     else:
#         temp_grid = np.copy(grid)
#         do_move(temp_grid, move, player)
#         moves, move_type = legal_move(temp_grid, -player)
#         if move_type == 'empty':
#             scores = [assess_move(temp_grid, next_move,-player, depth-1) for next_move in moves]
#         if move_type == 'cap':
#             scores = [assess_cap(temp_grid, next_move, -player, depth-1) for next_move in moves]
#         if player == 1:
#             score = np.min(scores)
#         elif player == -1:
#             score = np.max(scores)
#         print('empty')
#         print('player:',player, 'depth:',depth, '\n scores:',scores, 'score',score)
#         return score
        

In [8]:
# grid[5,2] = 0
# grid[4,1] = -1
# grid[2,1] = 0
# grid[3,0] = 1

# grid = np.array([[ 0,  0,  0,  0,  0,  0,  0,  0],
#                  [-1,  0,  0,  0,  0,  0,  0,  0],
#                  [ 0,  0,  0,  1,  0,  1,  0,  1],
#                  [ 0,  0,  0,  0,  0,  0,  1,  0],
#                  [ 0,  0,  0,  0,  0,  0,  0,  0],
#                  [ 0,  0,  0,  0,  0,  0,  0,  0],
#                  [ 0, -1,  0,  0,  0,  0,  0,  0],
#                  [-1,  0, -1,  0,  0,  0, -1,  0]])

# plt.imshow(grid)
# plt.show()
# # # for i in range(10):
# # #     max_arg, max_score, move, move_type = pick_move(grid, -1, 3)
# # pick_move(grid,-1,1)
# for i in range(10):
#     print(i)
#     time_1 = time.time()
#     print(pick_move(grid, -1, i+1))
#     time_2 = time.time()
#     print(time_2-time_1)

# cProfile.run('pick_move(grid, -1, 7)')

0
(array([[5, 2, 4, 1],
       [5, 4, 4, 3],
       [5, 6, 4, 5],
       [5, 0, 4, 1],
       [5, 2, 4, 3],
       [5, 4, 4, 5],
       [5, 6, 4, 7]]), array([0., 0., 0., 0., 0., 0., 0.]), 'empty', 0)
0.004217863082885742
1
(array([[5, 2, 4, 1],
       [5, 4, 4, 3],
       [5, 6, 4, 5],
       [5, 0, 4, 1],
       [5, 2, 4, 3],
       [5, 4, 4, 5],
       [5, 6, 4, 7]]), array([0., 0., 0., 0., 0., 0., 0.]), 'empty', 0)
0.006005764007568359
2
(array([[5, 2, 4, 1],
       [5, 4, 4, 3],
       [5, 6, 4, 5],
       [5, 0, 4, 1],
       [5, 2, 4, 3],
       [5, 4, 4, 5],
       [5, 6, 4, 7]]), array([0., 0., 0., 0., 0., 0., 0.]), 'empty', 0)
0.034894704818725586
3
(array([[5, 2, 4, 1],
       [5, 4, 4, 3],
       [5, 6, 4, 5],
       [5, 0, 4, 1],
       [5, 2, 4, 3],
       [5, 4, 4, 5],
       [5, 6, 4, 7]]), array([0., 0., 0., 0., 0., 0., 0.]), 'empty', 0)
0.21925735473632812
4
(array([[5, 2, 4, 1],
       [5, 4, 4, 3],
       [5, 6, 4, 5],
       [5, 0, 4, 1],
       [5, 2, 4, 3],
     

KeyboardInterrupt: 

In [ ]:
turns = 100
checkers = np.zeros((8,8,turns))
game_scores = np.zeros(turns)
turn =-1

for i in range(turns): 
    time_1 = time.time()
    plt.imshow(grid)
    plt.show()
    if turn == -1:
        moves, scores, move_type, best_move_arg = pick_move(grid, turn, 8)
    elif turn == 1:
        moves, scores, move_type, best_move_arg = pick_move(grid, turn, 2)
    game_scores[i] = scores[best_move_arg]
    print(moves, scores)
    print(grid)
    do_move(grid, moves[best_move_arg], turn)
    time_2 = time.time()
    if move_type != 'double_cap':
        turn *= -1
    checkers[:,:,i] = np.copy(grid)
    print(time_2-time_1)

In [ ]:
# def f(i):
#     plt.pcolor(checkers[:,:,i])
# ipy.interact(f,i=ipy.IntSlider(min=0,max=50), continuous_update=False)

In [ ]:
plt.plot(game_scores)
plt.show()